In [1]:
import os
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import shutil
from collections import defaultdict
from collections import Counter
from datetime import datetime

In [2]:
# Criar a pasta CSVs se não existir
pasta_destino = '/content/CSVs/'
os.makedirs(pasta_destino, exist_ok=True)

print(f"A pasta {pasta_destino} foi criada ou já existe.")

# Novo caminho para a pasta 'processados'
novo_local_processados = '/content/processados/'  # Novo local onde a pasta 'processados' ficará

# URL base da API do GitHub para acessar o repositório
url_base = "https://api.github.com/repos/PedroCosDi/estudos_ia/contents/aprendizado_supervisionado/queimadas_serie_temporal/dados/brutos/metereologicos"

# Função para buscar e listar todos os arquivos CSV em subpastas do diretório 'metereologicos'
def listar_arquivos_git(url):
    arquivos_csv = []

    # Obter os arquivos e pastas do diretório especificado
    response = requests.get(url)
    if response.status_code == 200:
        conteudo = response.json()
        for item in conteudo:
            if item['type'] == 'file' and item['name'].endswith('.CSV'):
                # Se for um arquivo CSV, adicionar o URL de download
                arquivos_csv.append(item['download_url'])
            elif item['type'] == 'dir':
                # Se for uma pasta, chamar recursivamente para listar os arquivos dessa pasta
                arquivos_csv.extend(listar_arquivos_git(item['url']))
        return arquivos_csv
    else:
        print(f"Erro ao acessar a API do GitHub: {response.status_code}")
        return []

# Listar todos os arquivos CSV nas subpastas dentro de 'metereologicos'
arquivos_csv = listar_arquivos_git(url_base)

# Função para baixar e processar o arquivo CSV
def baixar_e_processar_csv(url, pasta_destino):
    try:
        # Obter o conteúdo do arquivo da URL
        response = requests.get(url)

        # Verificar se a requisição foi bem-sucedida
        if response.status_code == 200:
            nome_arquivo = url.split('/')[-1]  # Extrair o nome do arquivo da URL
            caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)

            # Salvar o arquivo na pasta CSVs
            with open(caminho_arquivo, 'wb') as f:
                f.write(response.content)

            # Processar o arquivo CSV
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1', skiprows=8)  # Pular cabeçalho

            # Padronizar os nomes das colunas para letras maiúsculas
            df.columns = [col.strip().upper() for col in df.columns]

            # Colunas a serem removidas
            colunas_para_remover = ['UNNAMED: 19', 'VENTO, DIREÇÃO HORARIA (GR) (° (GR))']
            df = df.drop(columns=colunas_para_remover)

            # Salvar o arquivo processado na pasta 'processados' no novo local
            os.makedirs(novo_local_processados, exist_ok=True)
            caminho_arquivo_processado = os.path.join(novo_local_processados, nome_arquivo)
            df.to_csv(caminho_arquivo_processado, index=False, sep=';', encoding='latin-1')

            print(f"Arquivo {nome_arquivo} baixado e processado com sucesso.")
        else:
            print(f"Erro ao baixar o arquivo {url}. Código de status: {response.status_code}")
    except Exception as e:
        print(f"Ocorreu um erro ao baixar e processar o arquivo {url}: {e}")

# Função para processar os arquivos usando multithreading
def baixar_e_processar_em_lote(arquivos_csv, pasta_destino):
    with ThreadPoolExecutor(max_workers=20) as executor:
        # Distribui as tarefas para threads
        executor.map(lambda url: baixar_e_processar_csv(url, pasta_destino), arquivos_csv)

# Baixar e processar os arquivos CSV encontrados usando multithreading
baixar_e_processar_em_lote(arquivos_csv, pasta_destino)

# Deletar a pasta CSVs e manter apenas a pasta 'processados'
shutil.rmtree(pasta_destino)  # Remove a pasta CSVs
print(f"A pasta {pasta_destino} foi deletada.")

# A pasta 'processados' permanece no novo local
print(f"A pasta 'processados' foi mantida em {novo_local_processados}.")

A pasta /content/CSVs/ foi criada ou já existe.
Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2019_A_31-12-2019.CSV baixado e processado com sucesso.
Arquivo INMET_CO_GO_A012_LUZIANIA_01-01-2019_A_31-12-2019.CSV baixado e processado com sucesso.
Arquivo INMET_CO_GO_A025_RIO%20VERDE_01-01-2019_A_31-12-2019.CSV baixado e processado com sucesso.
Arquivo INMET_CO_GO_A023_CAIAPONIA_01-01-2019_A_31-12-2019.CSV baixado e processado com sucesso.
Arquivo INMET_CO_DF_A042_BRAZLANDIA_01-01-2019_A_31-12-2019.CSV baixado e processado com sucesso.
Arquivo INMET_CO_GO_A015_ITAPACI_01-01-2019_A_31-12-2019.CSV baixado e processado com sucesso.
Arquivo INMET_CO_GO_A011_SAO%20SIMAO_01-01-2019_A_31-12-2019.CSV baixado e processado com sucesso.
Arquivo INMET_CO_GO_A002_GOIANIA_01-01-2019_A_31-12-2019.CSV baixado e processado com sucesso.Arquivo INMET_CO_GO_A024_ALTO%20PARAISO%20DE%20GOIAS_01-01-2019_A_31-12-2019.CSV baixado e processado com sucesso.

Arquivo INMET_CO_DF_A047_PARANOA%20(COOPA-DF)_01-01-2019_A_31

In [3]:
# Carregar o arquivo
df = pd.read_csv('/content/processados/INMET_CO_DF_A001_BRASILIA_01-01-2019_A_31-12-2019.CSV', sep=';', encoding='latin-1')

# Exibir informações do arquivo
print(df.head())
print("\nInfo do DataFrame:")
df.info()

         DATA  HORA UTC PRECIPITAÇÃO TOTAL, HORÁRIO (MM)  \
0  2019/01/01  0000 UTC                                1   
1  2019/01/01  0100 UTC                                0   
2  2019/01/01  0200 UTC                                0   
3  2019/01/01  0300 UTC                               ,4   
4  2019/01/01  0400 UTC                                0   

  PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (MB)  \
0                                                887      
1                                              888,1      
2                                              888,2      
3                                              887,6      
4                                                887      

  PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (MB)  \
0                                             887   
1                                           888,1   
2                                           888,3   
3                                           888,2   
4                  

In [4]:
# Caminho para a pasta onde os arquivos CSV estão armazenados
folder_path = '/content/processados/'

# Listar todos os arquivos na pasta
arquivos = [arq for arq in os.listdir(folder_path) if arq.endswith('.CSV')]

# Contar o número de arquivos
num_arquivos = len(arquivos)

# Exibir o número de arquivos encontrados
print(f"Total de arquivos CSV na pasta: {num_arquivos}")

Total de arquivos CSV na pasta: 3465


In [5]:
# Listar todos os arquivos na pasta
arquivos = [arq for arq in os.listdir(folder_path) if arq.endswith('.CSV')]

# Contar a frequência de cada arquivo
contador = Counter(arquivos)

# Identificar arquivos com nome repetido
arquivos_repetidos = {arq: count for arq, count in contador.items() if count > 1}

# Exibir arquivos repetidos
if arquivos_repetidos:
    print("Arquivos com nomes repetidos encontrados:")
    for arq, count in arquivos_repetidos.items():
        print(f"{arq} - {count} ocorrências")
else:
    print("Não há arquivos com nomes repetidos.")

Não há arquivos com nomes repetidos.


In [6]:
# Listar todos os arquivos na pasta
arquivos = [arq for arq in os.listdir(folder_path) if arq.endswith('.CSV')]

# Função para extrair o nome da estação e o ano do nome do arquivo
def extrair_estacao_ano(nome_arquivo):
    partes = nome_arquivo.split('_')
    estacao = '_'.join(partes[3:5])  # Combina o código e o nome da estação
    ano = partes[-1].split('-')[-1][:4]  # Pega o ano no final do nome
    return estacao, int(ano)

# Organizar os arquivos por estação
stations = {}

# Classificar os arquivos por estação
for file in arquivos:
    station_name, year = extrair_estacao_ano(file)
    if station_name not in stations:
        stations[station_name] = []
    stations[station_name].append(year)

# Intervalo de anos que precisamos verificar
anos_requeridos = set(range(2019, 2025))  # Anos de 2019 a 2024

# Função para verificar se uma estação tem todos os anos
def verificar_estacao(station_name, years):
    anos_presentes = set(years)
    anos_faltando = anos_requeridos - anos_presentes
    if anos_faltando:
        return station_name
    return None

# Usar ThreadPoolExecutor para paralelizar o processo de verificação
with ThreadPoolExecutor(max_workers=20) as executor:
    # Enviar as tarefas de verificação para o executor
    resultados = list(executor.map(lambda x: verificar_estacao(x[0], x[1]), stations.items()))

# Identificar estações com anos faltando
stations_com_falta = [station for station in resultados if station]

# Excluir arquivos das estações que estão com anos faltando
for file in arquivos:
    station_name, _ = extrair_estacao_ano(file)
    if station_name in stations_com_falta:
        caminho_completo = os.path.join(folder_path, file)
        os.remove(caminho_completo)
        print(f"Arquivo deletado: {caminho_completo}")

print("Processo concluído. Arquivos das estações com anos faltando foram deletados.")

Arquivo deletado: /content/processados/INMET_SE_SP_A764_BEBDOURO_01-01-2020_A_31-12-2020.CSV
Arquivo deletado: /content/processados/INMET_CO_MS_A757_BELA%20VISTA_01-01-2019_A_31-12-2019.CSV
Arquivo deletado: /content/processados/INMET_S_RS_A808_TORRES_01-01-2023_A_31-12-2023.CSV
Arquivo deletado: /content/processados/INMET_S_PR_A822_NOVA%20TEBAS_01-01-2019_A_31-12-2019.CSV
Arquivo deletado: /content/processados/INMET_SE_MG_A521_PAMPULHA_01-01-2019_A_31-12-2019.CSV
Arquivo deletado: /content/processados/INMET_CO_MS_S714_PEDRO%20GOMES_01-01-2020_A_31-12-2020.CSV
Arquivo deletado: /content/processados/INMET_SE_ES_A623_NOVA%20VENECIA_01-01-2019_A_31-12-2019.CSV
Arquivo deletado: /content/processados/INMET_CO_MS_S703_BANDEIRANTES_01-01-2019_A_31-12-2019.CSV
Arquivo deletado: /content/processados/INMET_CO_MS_S704_BONITO_01-01-2021_A_31-12-2021.CSV
Arquivo deletado: /content/processados/INMET_SE_SP_A713_SOROCABA_01-01-2020_A_31-12-2020.CSV
Arquivo deletado: /content/processados/INMET_SE_SP_A7

In [7]:
# Listar todos os arquivos na pasta
arquivos = [arq for arq in os.listdir(folder_path) if arq.endswith('.CSV')]

# Contar o número de arquivos
num_arquivos = len(arquivos)

# Exibir o número de arquivos encontrados
print(f"Total de arquivos CSV na pasta: {num_arquivos}")

Total de arquivos CSV na pasta: 3282


In [8]:
output_folder = '/content/concatenados/'
os.makedirs(output_folder, exist_ok=True)

# Função para extrair o nome da estação, estado e o ano do nome do arquivo
def extrair_estacao_ano_estado(nome_arquivo):
    partes = nome_arquivo.split('_')
    estacao = '_'.join(partes[3:5])  # Combina o código e o nome da estação
    estado = partes[2]  # O estado está na terceira parte do nome
    ano = partes[-1].split('-')[-1][:4]  # Pega o ano no final do nome
    return estacao, estado, int(ano)

# Organizar os arquivos por estação e estado
stations = {}

# Classificar os arquivos por estação e estado
for file in arquivos:
    station_name, estado, year = extrair_estacao_ano_estado(file)
    if (station_name, estado) not in stations:
        stations[(station_name, estado)] = []
    stations[(station_name, estado)].append(file)

# Função para concatenar arquivos de uma estação e estado
def concatenar_estacao_estado(station_name, estado, files):
    # Ordenar os arquivos pelos anos
    files.sort(key=lambda f: extrair_estacao_ano_estado(f)[2])

    # Concatenar os arquivos em um único DataFrame
    dataframes = []
    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_csv(file_path, encoding='latin1', sep=';', on_bad_lines='error', engine='python')
        except Exception as e:
            print(f"Erro ao ler o arquivo {file_path}: {e}")
            continue
        dataframes.append(df)

    # Salvar os arquivos concatenados na pasta de saída, incluindo o estado no nome do arquivo
    if dataframes:
        concatenated_df = pd.concat(dataframes, ignore_index=True)
        output_file = os.path.join(output_folder, f"{station_name}_{estado}.csv")
        concatenated_df.to_csv(output_file, index=False, sep=';', encoding='latin-1')
        print(f"Arquivos concatenados para a estação {station_name} ({estado}): {output_file}")

# Usar ThreadPoolExecutor para paralelizar a concatenação
with ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(lambda item: concatenar_estacao_estado(item[0][0], item[0][1], item[1]), stations.items())

# Contar o número de arquivos na nova pasta
num_arquivos_concatenados = len(os.listdir(output_folder))
print(f"Número de arquivos na pasta concatenados: {num_arquivos_concatenados}")

# Validar se o número de arquivos está correto
num_esperado = len(arquivos) // 6
if num_arquivos_concatenados == num_esperado:
    print(f"Processo concluído com sucesso! Total de arquivos concatenados: {num_arquivos_concatenados}")
else:
    print(f"Atenção: número de arquivos concatenados ({num_arquivos_concatenados}) não corresponde ao esperado ({num_esperado}).")

Arquivos concatenados para a estação A043_CAMPOS%20LINDOS (TO): /content/concatenados/A043_CAMPOS%20LINDOS_TO.csv
Arquivos concatenados para a estação A421_BREJO%20GRANDE (SE): /content/concatenados/A421_BREJO%20GRANDE_SE.csv
Arquivos concatenados para a estação A108_CRUZEIRO%20DO%20SUL (AC): /content/concatenados/A108_CRUZEIRO%20DO%20SUL_AC.csv
Arquivos concatenados para a estação A117_COARI (AM): /content/concatenados/A117_COARI_AM.csv
Arquivos concatenados para a estação A821_JOAQUIM%20TAVORA (PR): /content/concatenados/A821_JOAQUIM%20TAVORA_PR.csv
Arquivos concatenados para a estação A721_DOURADOS (MS): /content/concatenados/A721_DOURADOS_MS.csv
Arquivos concatenados para a estação A374_CORRENTE (PI): /content/concatenados/A374_CORRENTE_PI.csv
Arquivos concatenados para a estação A432_BURITIRAMA (BA): /content/concatenados/A432_BURITIRAMA_BA.csv
Arquivos concatenados para a estação A839_PASSO%20FUNDO (RS): /content/concatenados/A839_PASSO%20FUNDO_RS.csv
Arquivos concatenados para a

In [9]:
# Caminho para a pasta a ser removida
folder_to_remove = '/content/processados/'

# Remover a pasta e seu conteúdo
shutil.rmtree(folder_to_remove)

print(f"A pasta {folder_to_remove} foi removida com sucesso!")

A pasta /content/processados/ foi removida com sucesso!


In [10]:
# Ler o arquivo
df = pd.read_csv('/content/concatenados/A011_SAO%20SIMAO_GO.csv', sep=';', encoding='latin-1')

# Exibir informações, primeiras e últimas linhas para verificar o arquivo
print(df.head())
print(df.tail())
print("\nInfo do DataFrame:")
df.info()

         DATA  HORA UTC PRECIPITAÇÃO TOTAL, HORÁRIO (MM)  \
0  2019/01/01  0000 UTC                                0   
1  2019/01/01  0100 UTC                                0   
2  2019/01/01  0200 UTC                                0   
3  2019/01/01  0300 UTC                                0   
4  2019/01/01  0400 UTC                                0   

  PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (MB)  \
0                                              954,8      
1                                              955,4      
2                                              955,7      
3                                              955,7      
4                                              955,1      

  PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (MB)  \
0                                           954,8   
1                                           955,4   
2                                           955,7   
3                                           955,9   
4                  

In [11]:
# Caminho para as pastas de entrada e saída
pasta_entrada = '/content/concatenados/'
pasta_saida = '/content/concatenados_processados/'
os.makedirs(pasta_saida, exist_ok=True)  # Criar a pasta de saída, se não existir

# Listar todos os arquivos CSV na pasta de entrada
arquivos_csv = [f for f in os.listdir(pasta_entrada) if f.endswith('.csv')]

def processar_arquivo(arquivo):
    try:
        print(f"Processando arquivo: {arquivo}")
        # Caminho completo do arquivo
        caminho_arquivo = os.path.join(pasta_entrada, arquivo)

        # Ler o arquivo CSV
        df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')

        # Transformações
        df['DATA'] = pd.to_datetime(df['DATA'] + ' ' + df['HORA UTC'], format='%Y/%m/%d %H%M UTC')
        df = df.drop(columns=['HORA UTC'])
        df = df.replace({',': '.'}, regex=True)
        df.set_index('DATA', inplace=True)
        df = df.apply(pd.to_numeric, errors='coerce')
        df = df.interpolate(method='linear')
        df = df.resample('ME').mean()

        # Extração da sigla do estado para o nome do arquivo
        uf = arquivo.split('_')[-1].split('.')[0].upper()  # Última parte do nome do arquivo antes do '.csv'
        caminho_arquivo_saida = os.path.join(pasta_saida, f"{uf}.pkl")

        # Salvar como Pickle
        df.to_pickle(caminho_arquivo_saida)
        print(f"Arquivo processado e salvo em: {caminho_arquivo_saida}\n")
    except Exception as e:
        print(f"Erro ao processar o arquivo {arquivo}: {e}")

# Processar arquivos em paralelo com ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(processar_arquivo, arquivos_csv)

print("Processamento concluído para todos os arquivos.")

Processando arquivo: A931_SANTO%20ANTONIO%20DO%20LESTE_MT.csv
Processando arquivo: A934_ALTO%20TAQUARI_MT.csv
Processando arquivo: A303_MACEIO_AL.csvProcessando arquivo: A372_IPANGUACU_RN.csv

Processando arquivo: A251_ITAUBAL_AP.csv
Processando arquivo: A016_JATAI_GO.csv
Processando arquivo: A340_APODI_RN.csv
Processando arquivo: A616_SAO%20MATEUS_ES.csv
Processando arquivo: A899_Santa%20Vitoria%20do%20Palmar%20-%20Barra%20do%20Chui_RS.csv
Processando arquivo: A311_FLORIANO_PI.csv
Processando arquivo: A533_GUANHAES_MG.csv
Processando arquivo: A706_CAMPOS%20DO%20JORDAO_SP.csv
Processando arquivo: A031_SAO%20MIGUEL%20DO%20ARAGUAIA_GO.csv
Processando arquivo: A724_CORUMBA_MS.csv
Processando arquivo: A667_SAQUAREMA%20-%20SAMPAIO%20CORREIA_RJ.csv
Processando arquivo: A234_TUCUMA_PA.csvProcessando arquivo: A881_DOM%20PEDRITO_RS.csv

Processando arquivo: A763_MARILIA_SP.csv
Processando arquivo: A233_SANTANA%20DO%20ARAGUAIA_PA.csvProcessando arquivo: A837_SOLEDADE_RS.csv



<ipython-input-11-e3fbd76427a8>:16: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo processado e salvo em: /content/concatenados_processados/AP.pkl

Processando arquivo: A727_LINS_SP.csv
Arquivo processado e salvo em: /content/concatenados_processados/PI.pkl

Processando arquivo: A230_SERRA%20DOS%20CARAJAS_PA.csv
Arquivo processado e salvo em: /content/concatenados_processados/MT.pkl

Processando arquivo: A429_BARRA_BA.csv
Arquivo processado e salvo em: /content/concatenados_processados/RN.pkl

Processando arquivo: A036_CRISTALINA_GO.csv
Arquivo processado e salvo em: /content/concatenados_processados/GO.pkl

Processando arquivo: A858_XANXERE_SC.csv
Arquivo processado e salvo em: /content/concatenados_processados/MS.pkl

Processando arquivo: A910_APIACAS_MT.csv
Arquivo processado e salvo em: /content/concatenados_processados/AL.pkl

Processando arquivo: A523_PATROCINIO_MG.csv


<ipython-input-11-e3fbd76427a8>:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo processado e salvo em: /content/concatenados_processados/PA.pkl

Processando arquivo: A831_QUARAI_RS.csv
Arquivo processado e salvo em: /content/concatenados_processados/MT.pkl

Processando arquivo: A252_DOM%20ELISEU_PA.csv
Arquivo processado e salvo em: /content/concatenados_processados/SP.pkl

Processando arquivo: A512_ITUIUTABA_MG.csv
Arquivo processado e salvo em: /content/concatenados_processados/ES.pkl

Processando arquivo: A456_SALVADOR%20(RADIO%20FAROL)_BA.csv
Arquivo processado e salvo em: /content/concatenados_processados/PA.pkl

Processando arquivo: A369_QUIXADA_CE.csv
Arquivo processado e salvo em: /content/concatenados_processados/RS.pkl

Processando arquivo: A832_SAO%20GABRIEL_RS.csv
Arquivo processado e salvo em: /content/concatenados_processados/RN.pkl

Processando arquivo: A887_CAPAO%20DO%20LEAO%20(PELOTAS)_RS.csv
Arquivo processado e salvo em: /content/concatenados_processados/RS.pkl

Processando arquivo: A246_MINA%20DO%20PALITO_PA.csv
Arquivo processado e sal

...

Arquivo processado e salvo em: /content/concatenados_processados/SP.pkl

Processando arquivo: A220_BACABAL_MA.csv
Arquivo processado e salvo em: /content/concatenados_processados/SP.pkl

Processando arquivo: A732_SAO%20GABRIEL%20DO%20OESTE_MS.csv
Arquivo processado e salvo em: /content/concatenados_processados/BA.pkl

Processando arquivo: A324_TAUA_CE.csv
Arquivo processado e salvo em: /content/concatenados_processados/AP.pkl

Processando arquivo: A224_ESTREITO_MA.csv
Arquivo processado e salvo em: /content/concatenados_processados/RS.pkl

Processando arquivo: A743_RIO%20BRILHANTE_MS.csv


<ipython-input-11-e3fbd76427a8>:16: DtypeWarning: Columns (8,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo processado e salvo em: /content/concatenados_processados/RJ.pkl

Processando arquivo: A428_SENHOR%20DO%20BONFIM_BA.csv
Arquivo processado e salvo em: /content/concatenados_processados/MG.pkl

Processando arquivo: A102_PARQUE%20ESTADUAL%20CHANDLESS_AC.csv
Arquivo processado e salvo em: /content/concatenados_processados/PR.pkl

Processando arquivo: A121_ITACOATIARA_AM.csv
Arquivo processado e salvo em: /content/concatenados_processados/RS.pkl

Processando arquivo: A761_SONORA_MS.csv
Arquivo processado e salvo em: /content/concatenados_processados/TO.pkl

Processando arquivo: A893_ENCRUZILHADA%20DO%20SUL_RS.csv
Arquivo processado e salvo em: /content/concatenados_processados/BA.pkl

Processando arquivo: A728_TAUBATE_SP.csv
Arquivo processado e salvo em: /content/concatenados_processados/RN.pkl

Processando arquivo: A920_JUINA_MT.csv
Arquivo processado e salvo em: /content/concatenados_processados/AM.pkl

Processando arquivo: A334_MONTEIRO_PB.csv
Arquivo processado e salvo em: /con

<ipython-input-11-e3fbd76427a8>:16: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo processado e salvo em: /content/concatenados_processados/CE.pkl

Processando arquivo: A525_SACRAMENTO_MG.csv
Arquivo processado e salvo em: /content/concatenados_processados/MA.pkl

Processando arquivo: A607_CAMPOS%20DOS%20GOYTACAZES_RJ.csv
Arquivo processado e salvo em: /content/concatenados_processados/MG.pkl

Processando arquivo: A765_BERTIOGA_SP.csv
Arquivo processado e salvo em: /content/concatenados_processados/RS.pkl

Processando arquivo: A836_JAGUARAO_RS.csv
Arquivo processado e salvo em: /content/concatenados_processados/MG.pkl

Processando arquivo: A227_SOURE_PA.csv
Arquivo processado e salvo em: /content/concatenados_processados/MS.pkl

Processando arquivo: A627_NITEROI_RJ.csv
Arquivo processado e salvo em: /content/concatenados_processados/MS.pkl

Processando arquivo: A013_ARAGARCAS_GO.csv
Arquivo processado e salvo em: /content/concatenados_processados/MA.pkl

Arquivo processado e salvo em: /content/concatenados_processados/BA.pkl

Arquivo processado e salvo em: /c

In [12]:
# Ler o arquivo Pickle
df = pd.read_pickle('/content/concatenados_processados/DF.pkl')

# Exibir o head e info do DataFrame
print(df.head())
print("\nInfo do DataFrame:")
print(df.info())

            PRECIPITAÇÃO TOTAL, HORÁRIO (MM)  \
DATA                                           
2019-01-31                          0.113710   
2019-02-28                          0.400595   
2019-03-31                          0.189247   
2019-04-30                          0.301389   
2019-05-31                          0.100806   

            PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (MB)  \
DATA                                                                
2019-01-31                                         905.383199       
2019-02-28                                         904.088988       
2019-03-31                                         904.843280       
2019-04-30                                         904.902917       
2019-05-31                                         905.536022       

            PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (MB)  \
DATA                                                          
2019-01-31                                       905.

In [13]:
pasta_processados = '/content/concatenados/'

if os.path.exists(pasta_processados):
    shutil.rmtree(pasta_processados)
    print("Pasta deletada com sucesso.")
else:
    print("A pasta não existe.")

Pasta deletada com sucesso.


In [14]:
# Caminho da pasta de entrada e saída
pasta_entrada = '/content/concatenados_processados/'
pasta_saida = '/content/concatenados_processados_estaduais/'

# Criar a pasta de saída se não existir
os.makedirs(pasta_saida, exist_ok=True)

# Listar todos os arquivos .pkl na pasta de entrada
arquivos_pkl = [f for f in os.listdir(pasta_entrada) if f.endswith('.pkl')]

# Dicionário para armazenar os DataFrames por estado
dados_estaduais = defaultdict(list)

# Processar cada arquivo pkl
for arquivo in arquivos_pkl:
    try:
        # Caminho completo do arquivo
        caminho_arquivo = os.path.join(pasta_entrada, arquivo)

        # Extrair a sigla do estado (parte do nome do arquivo)
        estado = arquivo.split('_')[-1].replace('.pkl', '')  # Exemplo: 'DF'

        # Carregar o arquivo pkl
        df = pd.read_pickle(caminho_arquivo)

        # Adicionar o DataFrame ao dicionário, agrupando por estado
        dados_estaduais[estado].append(df)

    except Exception as e:
        print(f"Erro ao processar o arquivo {arquivo}: {e}")

# Calcular a média por estado e salvar os arquivos finais
for estado, dfs in dados_estaduais.items():
    try:
        # Concatenar todos os DataFrames do mesmo estado
        df_estado = pd.concat(dfs)

        # Calcular a média mensal para as colunas numéricas
        df_estado = df_estado.resample('ME').mean()

        # Caminho para salvar o novo arquivo pkl (sem 'A001_')
        caminho_arquivo_saida = os.path.join(pasta_saida, f"{estado}.pkl")

        # Salvar o DataFrame estadual
        df_estado.to_pickle(caminho_arquivo_saida)

        print(f"Arquivo estadual {estado} processado e salvo em: {caminho_arquivo_saida}")

    except Exception as e:
        print(f"Erro ao salvar o arquivo estadual {estado}: {e}")

print("Processamento concluído para todos os estados.")

Arquivo estadual TO processado e salvo em: /content/concatenados_processados_estaduais/TO.pkl
Arquivo estadual MS processado e salvo em: /content/concatenados_processados_estaduais/MS.pkl
Arquivo estadual AM processado e salvo em: /content/concatenados_processados_estaduais/AM.pkl
Arquivo estadual AL processado e salvo em: /content/concatenados_processados_estaduais/AL.pkl
Arquivo estadual MT processado e salvo em: /content/concatenados_processados_estaduais/MT.pkl
Arquivo estadual GO processado e salvo em: /content/concatenados_processados_estaduais/GO.pkl
Arquivo estadual PB processado e salvo em: /content/concatenados_processados_estaduais/PB.pkl
Arquivo estadual SE processado e salvo em: /content/concatenados_processados_estaduais/SE.pkl
Arquivo estadual SP processado e salvo em: /content/concatenados_processados_estaduais/SP.pkl
Arquivo estadual PI processado e salvo em: /content/concatenados_processados_estaduais/PI.pkl
Arquivo estadual RS processado e salvo em: /content/concaten

In [15]:
# Caminho do arquivo estadual
caminho_arquivo = '/content/concatenados_processados_estaduais/DF.pkl'

# Ler e mostrar head e info
df = pd.read_pickle(caminho_arquivo)
print(df.head(), "\n")
print("\nInfo do DataFrame:")
print(df.info())

            PRECIPITAÇÃO TOTAL, HORÁRIO (MM)  \
DATA                                           
2019-01-31                          0.113710   
2019-02-28                          0.400595   
2019-03-31                          0.189247   
2019-04-30                          0.301389   
2019-05-31                          0.100806   

            PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (MB)  \
DATA                                                                
2019-01-31                                         905.383199       
2019-02-28                                         904.088988       
2019-03-31                                         904.843280       
2019-04-30                                         904.902917       
2019-05-31                                         905.536022       

            PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (MB)  \
DATA                                                          
2019-01-31                                       905.

In [16]:
pasta_processados = '/content/concatenados_processados/'

if os.path.exists(pasta_processados):
    shutil.rmtree(pasta_processados)
    print("Pasta deletada com sucesso.")
else:
    print("A pasta não existe.")

Pasta deletada com sucesso.


In [17]:
# Criar o intervalo de datas mensais que vai de janeiro de 2019 a novembro de 2024
all_months = pd.date_range(start='2019-01-01', end='2024-11-30', freq='ME')

# Caminho da pasta onde estão os arquivos Pickle
pasta_pickle = "/content/concatenados_processados_estaduais/"

# Iterar por todos os arquivos na pasta
for arquivo in os.listdir(pasta_pickle):
    if arquivo.endswith(".pkl"):  # Verificar se é um arquivo Pickle
        caminho_completo = os.path.join(pasta_pickle, arquivo)

        # Carregar o DataFrame do arquivo Pickle
        df = pd.read_pickle(caminho_completo)

        # Reindexar o DataFrame para garantir que todos os meses estejam presentes
        df = df.reindex(all_months)

        # Substituir os padrões 'HORÁRIO', 'NA HORA ANT.' e 'HORARIA' por 'MENSAL' nas colunas
        df.columns = [
            col.replace('HORÁRIO', 'MENSAL').replace('NA HORA ANT.', 'MENSAL').replace('HORARIA', 'MENSAL') for col in df.columns
        ]

        # Interpolação linear para preencher valores ausentes
        df = df.interpolate(method='linear', axis=0, limit_direction='both')

        # Salvar o DataFrame atualizado de volta no arquivo Pickle
        df.to_pickle(caminho_completo)

        # Exibir uma mensagem de confirmação para cada arquivo processado
        print(f"Atualizado e processado: {arquivo}")

Atualizado e processado: TO.pkl
Atualizado e processado: MS.pkl
Atualizado e processado: AM.pkl
Atualizado e processado: AL.pkl
Atualizado e processado: MT.pkl
Atualizado e processado: GO.pkl
Atualizado e processado: PB.pkl
Atualizado e processado: SE.pkl
Atualizado e processado: SP.pkl
Atualizado e processado: PI.pkl
Atualizado e processado: RS.pkl
Atualizado e processado: RN.pkl
Atualizado e processado: ES.pkl
Atualizado e processado: CE.pkl
Atualizado e processado: DF.pkl
Atualizado e processado: RO.pkl
Atualizado e processado: BA.pkl
Atualizado e processado: AP.pkl
Atualizado e processado: RJ.pkl
Atualizado e processado: PR.pkl
Atualizado e processado: AC.pkl
Atualizado e processado: MG.pkl
Atualizado e processado: SC.pkl
Atualizado e processado: PE.pkl
Atualizado e processado: PA.pkl
Atualizado e processado: RR.pkl
Atualizado e processado: MA.pkl


In [18]:
# Carregar o arquivo .pkl
estado_data = pd.read_pickle('/content/concatenados_processados_estaduais/AC.pkl')

# Exibir informações sobre o DataFrame do estado
print(estado_data.head())
print("\nInfo do DataFrame:")
estado_data.info()

            PRECIPITAÇÃO TOTAL, MENSAL (MM)  \
2019-01-31                         0.010526   
2019-02-28                         0.010526   
2019-03-31                         0.010526   
2019-04-30                         0.010526   
2019-05-31                         0.010526   

            PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, MENSAL (MB)  \
2019-01-31                                         990.680309      
2019-02-28                                         990.317485      
2019-03-31                                         990.426478      
2019-04-30                                         990.362569      
2019-05-31                                         991.984341      

            PRESSÃO ATMOSFERICA MAX.MENSAL (AUT) (MB)  \
2019-01-31                                 991.019960   
2019-02-28                                 990.650670   
2019-03-31                                 990.810013   
2019-04-30                                 990.716319   
2019-05-31             

In [19]:
# URL do arquivo CSV
url = 'https://github.com/PedroCosDi/estudos_ia/blob/main/aprendizado_supervisionado/queimadas_serie_temporal/dados/brutos/queimadas/queimadas_brasil.csv?raw=true'

# Caminho para salvar o arquivo temporariamente
caminho_arquivo = '/content/queimadas_brasil.csv'

# Baixar o arquivo CSV
response = requests.get(url)
if response.status_code == 200:
    with open(caminho_arquivo, 'wb') as file:
        file.write(response.content)
    print(f"Arquivo {caminho_arquivo} baixado com sucesso.")
else:
    print(f"Erro ao baixar o arquivo. Status code: {response.status_code}")


# Ler o arquivo CSV com o encoding e separador corretos
df_queimadas = pd.read_csv(caminho_arquivo, sep=';', encoding='utf-8-sig', engine='python')

# Exibir as primeiras linhas para verificação
print(df_queimadas.head())
print("\nInfo do DataFrame:")
df_queimadas.info()

Arquivo /content/queimadas_brasil.csv baixado com sucesso.
      date                                      class  focuses         uf
0  2021/12  Fogo em áreas de desmatamento consolidado       18  TOCANTINS
1  2019/04  Fogo em áreas de desmatamento consolidado      119  TOCANTINS
2  2024/11      Fogo em áreas de desmatamento recente      183  TOCANTINS
3  2019/04      Fogo em áreas de desmatamento recente       49  TOCANTINS
4  2019/04          Fogo em áreas de vegetação nativa       39  TOCANTINS

Info do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8132 entries, 0 to 8131
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     8132 non-null   object
 1   class    8132 non-null   object
 2   focuses  8132 non-null   int64 
 3   uf       8132 non-null   object
dtypes: int64(1), object(3)
memory usage: 254.2+ KB


In [20]:
# Remover a coluna 'class'
df_queimadas = df_queimadas.drop(columns=['class'])

# Transformar a coluna 'date' em índice
df_queimadas['date'] = pd.to_datetime(df_queimadas['date'], format='%Y/%m') + pd.offsets.MonthEnd(0)  # Converter para datetime
df_queimadas.set_index('date', inplace=True)

# Agrupar por 'date' e 'uf' e somar os valores de 'focuses'
df_queimadas = df_queimadas.groupby([df_queimadas.index, 'uf']).agg({'focuses': 'sum'})

# Renomear a coluna 'focuses' para 'FOCOS DE QUEIMADA'
df_queimadas.rename(columns={'focuses': 'FOCOS DE QUEIMADA'}, inplace=True)

# Resetar o índice para garantir um único índice com a data
df_queimadas.reset_index(inplace=True)

# Exibir os resultados
print(df_queimadas.info())
print("\nInfo do DataFrame:")
print(df_queimadas.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1878 entries, 0 to 1877
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               1878 non-null   datetime64[ns]
 1   uf                 1878 non-null   object        
 2   FOCOS DE QUEIMADA  1878 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 44.1+ KB
None

Info do DataFrame:
        date        uf  FOCOS DE QUEIMADA
0 2019-01-31      ACRE                  1
1 2019-01-31   ALAGOAS                 32
2 2019-01-31     AMAPÁ                  4
3 2019-01-31  AMAZONAS                 35
4 2019-01-31     BAHIA                212


In [21]:
# Criação de um dicionário para armazenar os DataFrames individuais por estado
df_estados = {}

# Criar o intervalo de datas mensais que vai de janeiro de 2019 a dezembro de 2024
all_months = pd.date_range(start='2019-01-01', end='2024-11-30', freq='ME')

# Iterar por cada estado único e criar o DataFrame correspondente
for estado in df_queimadas['uf'].unique():
    df_estado = df_queimadas[df_queimadas['uf'] == estado].copy()  # Filtrar dados do estado
    df_estado.set_index('date', inplace=True)  # Definir 'date' como índice
    df_estado.drop(columns=['uf'], inplace=True)  # Remover a coluna 'uf'

    # Remover a última instância (última linha) de cada DataFrame
    df_estado = df_estado.iloc[:-1]

    # Reindexar o DataFrame para garantir que todos os meses estejam presentes
    df_estado = df_estado.reindex(all_months)

    # Interpolação linear para preencher valores ausentes
    df_estado = df_estado.interpolate(method='linear')

    # Armazenar o DataFrame no dicionário com o nome do estado
    df_estados[estado] = df_estado

# Acessar o DataFrame de um estado específico e ver algumas informações
df_bahia = df_estados['BAHIA']
print(df_bahia.tail())
print("\nInfo do DataFrame:")
df_bahia.info()

            FOCOS DE QUEIMADA
2024-07-31                655
2024-08-31                949
2024-09-30               2756
2024-10-31               2198
2024-11-30                324

Info do DataFrame:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71 entries, 2019-01-31 to 2024-11-30
Freq: ME
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   FOCOS DE QUEIMADA  71 non-null     int64
dtypes: int64(1)
memory usage: 1.1 KB


In [22]:
# Dicionário para mapear nome completo para sigla
sigla_para_nome = {
    'ACRE': 'AC', 'ALAGOAS': 'AL', 'AMAPÁ': 'AP', 'AMAZONAS': 'AM', 'BAHIA': 'BA',
    'CEARÁ': 'CE', 'DISTRITO FEDERAL': 'DF', 'ESPÍRITO SANTO': 'ES', 'GOIÁS': 'GO',
    'MARANHÃO': 'MA', 'MATO GROSSO': 'MT', 'MATO GROSSO DO SUL': 'MS', 'MINAS GERAIS': 'MG',
    'PARÁ': 'PA', 'PARAÍBA': 'PB', 'PERNAMBUCO': 'PE', 'PIAUÍ': 'PI', 'RIO DE JANEIRO': 'RJ',
    'RIO GRANDE DO NORTE': 'RN', 'RIO GRANDE DO SUL': 'RS', 'RONDÔNIA': 'RO', 'RORAIMA': 'RR',
    'SANTA CATARINA': 'SC', 'SÃO PAULO': 'SP', 'SERGIPE': 'SE', 'TOCANTINS': 'TO',
    'PARANÁ': 'PR'
}

# Caminho para a pasta de saída dos arquivos CSV
pasta_saida = '/content/concatenados_final/'

# Criar a pasta de saída, se não existir
os.makedirs(pasta_saida, exist_ok=True)

# Caminho onde estão os pickles (usando a sigla do estado)
pickle_path = "/content/concatenados_processados_estaduais/{}.pkl"

# Dicionário para armazenar os DataFrames combinados
df_combinados = {}

# Iterar por cada estado único
for estado_nome_completo, df_estado in df_estados.items():
    # Obter a sigla do estado do dicionário
    estado_sigla = sigla_para_nome.get(estado_nome_completo)

    if estado_sigla is not None:
        # Carregar o pickle correspondente ao estado (usando a sigla)
        df_pickle = pd.read_pickle(pickle_path.format(estado_sigla))

        # Juntar o DataFrame do estado com o pickle (assumindo que o índice de ambos seja 'date')
        df_combinado = df_pickle.join(df_estado[['FOCOS DE QUEIMADA']])  # 'focuses' é a coluna do df_queimadas

        # Armazenar o DataFrame combinado no dicionário
        df_combinados[estado_nome_completo] = df_combinado

        # Salvar o DataFrame combinado como CSV dentro da pasta de saída, com o nome adequado
        df_combinado.to_csv(os.path.join(pasta_saida, f'{estado_sigla}_final.csv'))

        # Exibir um exemplo para ver como ficou
        print(f"DataFrame combinado para {estado_nome_completo}:")
    else:
        print(f"Sigla para o estado {estado_nome_completo} não encontrada.")

DataFrame combinado para ACRE:
DataFrame combinado para ALAGOAS:
DataFrame combinado para AMAPÁ:
DataFrame combinado para AMAZONAS:
DataFrame combinado para BAHIA:
DataFrame combinado para CEARÁ:
DataFrame combinado para DISTRITO FEDERAL:
DataFrame combinado para ESPÍRITO SANTO:
DataFrame combinado para GOIÁS:
DataFrame combinado para MARANHÃO:
DataFrame combinado para MATO GROSSO:
DataFrame combinado para MATO GROSSO DO SUL:
DataFrame combinado para MINAS GERAIS:
DataFrame combinado para PARANÁ:
DataFrame combinado para PARAÍBA:
DataFrame combinado para PARÁ:
DataFrame combinado para PERNAMBUCO:
DataFrame combinado para PIAUÍ:
DataFrame combinado para RIO DE JANEIRO:
DataFrame combinado para RIO GRANDE DO NORTE:
DataFrame combinado para RIO GRANDE DO SUL:
DataFrame combinado para RONDÔNIA:
DataFrame combinado para RORAIMA:
DataFrame combinado para SANTA CATARINA:
DataFrame combinado para SERGIPE:
DataFrame combinado para SÃO PAULO:
DataFrame combinado para TOCANTINS:


In [23]:
# Carregar o arquivo CSV, definir a primeira coluna (índice 0) como índice e converter para datetime
df_ac = pd.read_csv('/content/concatenados_final/AC_final.csv', index_col=0, parse_dates=[0])

# Exibir head e info
print(df_ac.head())

print("\nInformações do DataFrame:")
print(df_ac.info())

            PRECIPITAÇÃO TOTAL, MENSAL (MM)  \
2019-01-31                         0.010526   
2019-02-28                         0.010526   
2019-03-31                         0.010526   
2019-04-30                         0.010526   
2019-05-31                         0.010526   

            PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, MENSAL (MB)  \
2019-01-31                                         990.680309      
2019-02-28                                         990.317485      
2019-03-31                                         990.426478      
2019-04-30                                         990.362569      
2019-05-31                                         991.984341      

            PRESSÃO ATMOSFERICA MAX.MENSAL (AUT) (MB)  \
2019-01-31                                 991.019960   
2019-02-28                                 990.650670   
2019-03-31                                 990.810013   
2019-04-30                                 990.716319   
2019-05-31             

In [24]:
# Deletar a pasta
pasta = '/content/concatenados_processados_estaduais'
if os.path.exists(pasta):
    shutil.rmtree(pasta)
    print(f'Pasta {pasta} deletada.')

# Deletar o arquivo CSV
arquivo_csv = '/content/queimadas_brasil.csv'
if os.path.exists(arquivo_csv):
    os.remove(arquivo_csv)
    print(f'Arquivo {arquivo_csv} deletado.')

Pasta /content/concatenados_processados_estaduais deletada.
Arquivo /content/queimadas_brasil.csv deletado.


In [25]:
# Caminho da pasta
pasta = '/content/concatenados_final/'

# Lista todos os arquivos na pasta
arquivos = os.listdir(pasta)

# Filtra os arquivos CSV
arquivos_csv = [arquivo for arquivo in arquivos if arquivo.endswith('.csv')]

# Cria um diretório temporário para os arquivos a serem baixados
zip_file = '/content/concatenados_final.zip'
shutil.make_archive(zip_file.replace('.zip', ''), 'zip', pasta)

# Link para download
from google.colab import files
files.download(zip_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>